## Modelo SIR en parareal (Julia)

   
\begin{align}
S_{t} & = - \beta S\cdot I  \\
I_{t} & = \beta S \cdot I - \nu I \\
R_{t} & = \nu I 
\end{align}

   
\begin{align}
S_{t} & = - \beta S\cdot I  \\
I_{t} & = \beta S \cdot I - \nu I \\
R(t) & = N - S(t)-I(t)
\end{align}


In [85]:
# Parámetros globales

const global N_ind = 103 # Número de individuos  
const global beta = 0.07
const global nu = 0.5

const global S0 = 100.
const global I0 = 3.
const global R0 = N_ind-S0-I0
const global U0 = [S0, I0, R0]

const global t_init = 0.
const global t_end  = 5. 
t_n_f = 1000000
const global t_n_c = 100

const global N_intervalos = 160;
const global T = LinRange(t_init, t_end, N_intervalos+1)

const global n_iter = 4 # Número de iteraciones de Parareal

SIR_1(S,I) = -beta*S*I
SIR_2(S,I) =  beta*S*I-nu*I
SIR_3(S,I) =  nu*I

SIR_3 (generic function with 1 method)

## Array de incógintas, $U[i,j,k]$

- Primer índice $i$: subintervalo
- Segundo índice $j$: iteración parareal
- Terecer ínidice $k$: incógnita
    * 1 -> S
    * 2 -> I
    * 3 -> R

## Método de Euler

In [86]:
# Resuelve el sistema de EDO U' = F(U,t), U(0)=U0 en el rango 
# de tiempo T = [t_0, ..., t_n] mediante el método de Euler
# Almacena la solución en U_sol
function Euler_SIR(U0, t_init, t_end, n_t)
    dt = (t_end-t_init)/n_t
    t = t_init
    
    #println("n_t=$n_t, t_init=$t_init, t_end=$t_end, dt=$dt")
    # U_sol = zeros(size(U0))
    S0, I0, R0 = U0
    S1, I1, R1 = U0
    for i=1:n_t
        # Denotamos: U=solución en la etapa actual, U0=sol. etapa anterior
        S1 = S0 + dt*SIR_1(S0, I0)
        I1 = I0 + dt*SIR_2(S0, I0)
        R1 = R0 + dt*SIR_3(S0, I0)

        # Preparamos siguiente iteración
        t += dt
        S0, I0, R0 = S1, I1, R1
        
        #println("Iter $i, t=$t, u=$U_sol, ex_sol=$(exp(t))")
    end
    return [S1, I1, R1]
end

Euler_SIR (generic function with 1 method)

 ## Método Parareal

In [87]:
@inline F(t1, t0, u0) =  Euler_SIR(u0, t0, t1, t_n_f)
@inline G(t1, t0, u0) =  Euler_SIR(u0, t0, t1, t_n_c)
#F!(t1, t0, u0, U_sol) =  Euler_SIR!(u0, t0, t1, t_n_f, U_sol)
#G!(t1, t0, u0, U_sol) =  Euler_SIR!(u0, t0, t1, t_n_c, U_sol)

G (generic function with 1 method)

In [88]:
function SIR_secuencial()
    U = Array{Float64,3}(undef, N_intervalos+1, n_iter+1, 3);
    Fn = Array{Float64,2}(undef, N_intervalos+1, 3);

    # 1.a) Inicialización (aproximción grosera)
    U[1,1,:] = U0
    for n=1:N_intervalos
        U[n+1,1,:] = G( T[n+1],T[n],U[n,1,:] )
    end
        
    # 1.b) Inicialización etapas parareal
    for k=1:n_iter
        U[1,k+1,:] = U0
    end

    # 2) Bucle parareal
    for k=1:n_iter
 
        # 2.a) Aproximación fina (paralela) en cada subintervalo
        @time @inbounds begin
        for n = 1:N_intervalos
            #F_sol[n,:] = F( T[n+1], T[n], U[n,k,:] )
            t0 = T[n]
            t1 = T[n+1]
            Unk = U[n,k,:]
            Fn[n,:] = F( t1, t0, Unk )
        end
        end
        
        # 2.b) Corrección secuencial
        for n = 1:N_intervalos
            U[n+1, k+1, :] = Fn[n,:] + G( T[n+1], T[n], U[n,k+1,:] ) - G( T[n+1], T[n], U[n,k,:] )
        end
    end
    
    return U
    
end

SIR_secuencial (generic function with 1 method)

In [89]:
function SIR_parareal()
    U = Array{Float64,3}(undef, N_intervalos+1, n_iter+1, 3);
    Fn = Array{Float64,2}(undef, N_intervalos+1, 3);
    Un = Array{Float64,2}(undef, N_intervalos+1, 3);

    # 1.a) Inicialización (aproximción grosera)
    U[1,1,:] = U0
    
    for n=1:N_intervalos
        U[n+1,1,:] = G( T[n+1],T[n],U[n,1,:] )
    end
        
    # 1.b) Inicialización etapas parareal
    @inbounds Threads.@threads for k=1:n_iter
        U[1,k+1,:] = U0
    end

    # 2) Bucle parareal
    for k=1:n_iter
 
        # 2.a) Aproximación fina (paralela) en cada subintervalo
        begin
        @inbounds Threads.@threads for n = 1:N_intervalos
            t0 = T[n]
            t1 = T[n+1]
            #Unk = U[n,k,:]
            Fn[n,:] = F( t1, t0, U[n,k,:])
        end
        end
        
        # 2.b) Corrección secuencial
        for n = 1:N_intervalos
            U[n+1, k+1, :] = Fn[n,:] + G( T[n+1], T[n], U[n,k+1,:] ) - G( T[n+1], T[n], U[n,k,:] )
        end

    end
    
    return U
    
end

SIR_parareal (generic function with 1 method)

In [81]:
@time U = SIR_secuencial();

  0.016145 seconds (803 allocations: 42.719 KiB)
  0.016093 seconds (800 allocations: 42.500 KiB)
  0.016094 seconds (800 allocations: 42.500 KiB)
  0.016092 seconds (800 allocations: 42.500 KiB)
  0.237322 seconds (207.02 k allocations: 10.773 MiB)


In [90]:
@time U = SIR_parareal(); 

  0.663107 seconds (279.54 k allocations: 14.689 MiB)


In [91]:
#import Pkg; Pkg.add("Plots")

using Plots

y = U[:,end,:]; # Susceptibles
x = T
plotly()
plot(x, y, label = ["Susceptibles" "Infectados" "Recuperados"], xlabel="Tiempo", ylabel="Nº individuos",title="Evolución epidemia",linewidth = 2)

<!DOCTYPE html>
 
 
 Plots.jl

## Comparación con Euler


In [9]:
N_total = N_intervalos*t_n_f
@time resultado = Euler_SIR(U0, t_init, t_end, N_total);

  0.001616 seconds (6 allocations: 320 bytes)


In [10]:
dif = resultado - U[end,end,:]
error_absoluto = sqrt( sum(dif.^2) ) 
error_relativo = error_absoluto / sqrt(sum(resultado.^2))
println("Diferencia entre el método de Euler y Parareal/Euler en la última iteración:")
println("Error absoluto: $error_absoluto.\nError relativo: $error_relativo")

Diferencia entre el método de Euler y Parareal/Euler en la última iteración:
Error absoluto: 9.882722195091865e-13.
Error relativo: 9.748653122478215e-15


# N_intervalos = núcleos

In [19]:
lista_tiempos_x = LinRange(100000,1000000,10)
lista_tiempos = Array{Float64,2}(undef, 10, 2);
for i=1:10
    t_n_f = 100000*i
    N_total = N_intervalos*t_n_f
    t1_l = @elapsed Euler_SIR(U0, t_init, t_end, N_total);
    t2_l = @elapsed SIR_parareal()
    lista_tiempos[i,1] = t1_l
    lista_tiempos[i,2] = t2_l
end
    

In [20]:
plotly()
plot(lista_tiempos_x, lista_tiempos, label=["Secuencial" "Paralelo"], title="N_intervalos = núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

### DIFERENCIA

In [43]:
diferencia_16 = lista_tiempos[:,1] - lista_tiempos[:,2] ;

In [44]:
plotly()
plot(lista_tiempos_x, diferencia_16, label="(Secuencial - Paralelo)", title="N_intervalos = núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

### DIFERENCIA RELATIVA

In [92]:
diferencia_relativa_16 = diferencia_16 ./ lista_tiempos[:,2] ;

In [93]:
plotly()
plot(lista_tiempos_x, diferencia_relativa_16, label="(Secuencial - Paralelo)", title="N_intervalos = núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

# N_intervalos = 2*núcleos

In [36]:
lista_tiempos_32 = Array{Float64,2}(undef, 10, 2);
for i=1:10
    t_n_f = 100000*i
    N_total = N_intervalos*t_n_f
    t1_l = @elapsed Euler_SIR(U0, t_init, t_end, N_total);
    t2_l = @elapsed SIR_parareal()
    lista_tiempos_32[i,1] = t1_l
    lista_tiempos_32[i,2] = t2_l
end
    

In [37]:
plotly()
plot(lista_tiempos_x, lista_tiempos_32, label=["Secuencial" "Paralelo"], title="N_intervalos = 2*núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

### DIFERENCIA

In [40]:
diferencia_32 = lista_tiempos_32[:,1] - lista_tiempos_32[:,2] ;

In [42]:
plotly()
plot(lista_tiempos_x, diferencia_32, label="(Secuencial - Paralelo)", title="N_intervalos = 2*núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

### DIFERENCIA RELATIVA

In [94]:
diferencia_relativa_32 = diferencia_32 ./ lista_tiempos_32[:,2] ;

In [95]:
plotly()
plot(lista_tiempos_x, diferencia_relativa_32, label="(Secuencial - Paralelo)", title="N_intervalos = 2*núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

# N_intervalos = 10*núcleos

In [58]:
lista_tiempos_160 = Array{Float64,2}(undef, 10, 2);
for i=1:10
    t_n_f = 100000*i
    N_total = N_intervalos*t_n_f
    t1_l = @elapsed Euler_SIR(U0, t_init, t_end, N_total);
    t2_l = @elapsed SIR_parareal()
    lista_tiempos_160[i,1] = t1_l
    lista_tiempos_160[i,2] = t2_l
end

In [59]:
plotly()
plot(lista_tiempos_x, lista_tiempos_160, label=["Secuencial" "Paralelo"], title="N_intervalos = 10*núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

### DIFERENCIA

In [60]:
diferencia_160 = lista_tiempos_160[:,1] - lista_tiempos_160[:,2] ;

In [61]:
plotly()
plot(lista_tiempos_x, diferencia_160, label="(Secuencial - Paralelo)", title="N_intervalos = 10*núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

### DIFERENCIA RELATIVA

In [96]:
diferencia_relativa_160 = diferencia_160 ./ lista_tiempos_160[:,2] ;

In [97]:
plotly()
plot(lista_tiempos_x, diferencia_relativa_160, label="(Secuencial - Paralelo)", title="N_intervalos = 10*núcleos", xlabel="Iteraciones finas", ylabel="Tiempo(s)")

<!DOCTYPE html>
 
 
 Plots.jl

In [105]:
comparacion = Array{Float64,2}(undef, 10,3)
comparacion[:,1], comparacion[:,2], comparacion[:,3] = diferencia_relativa_16, diferencia_relativa_32, diferencia_relativa_160

([0.6637021059954546, 1.5830945187911394, 1.7983161839219162, 1.8142522640908392, 1.9432936194418984, 2.013494289795133, 2.0187401857205614, 2.133422385818195, 2.0524284561405386, 2.4432427857181342], [1.1089441843570855, 1.7529290445215113, 1.9183032232101858, 1.9182182894147597, 1.6758493181770608, 2.3005361421934585, 2.3367577104040667, 2.3322798676868115, 2.473027382685274, 2.3877096085041343], [2.1460178501903253, 2.4755913893414467, 2.357906377224701, 2.643121244780945, 2.6254733286117307, 2.650720501466349, 2.5703338573960597, 2.749552370762375, 2.744469503212316, 2.76853992242977])

In [108]:
plotly()
plot(lista_tiempos_x, comparacion, label=["16" "32" "160"])

<!DOCTYPE html>
 
 
 Plots.jl